In [5]:
import pandas as pd
import mysql.connector as msql
from mysql.connector import Error
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import os.path
from selenium import webdriver
import pandas as pd
from pandas import concat
import glob
import time

MY FUNCTIONS

In [4]:
from func import functions
from dict import lists

In [3]:
class DATA_SOURCE:
    def ELO_source(season): # one argument - season. FORMAT: {start_season - end_season} "2022-2023"
        elo_list = lists.elo_dict(season)
        ELO = []
        for i in elo_list:
            web = f'http://elofootball.com/country.php?countryiso={i}&season={season}'
            ELO.append(web)
        print('WEB ELO lists complete')
        return ELO
    
    def soccer_stats_data(season, table, date): # two arguments. Season = end date. Table = overall| home/away | matches
            comp_list = lists.competition_list()
            overall_list = []
            home_away_list = []
            matches = []
            if date == 'historical':
                space = '_'
                for i in comp_list:
                    web = f'https://www.soccerstats.com/latest.asp?league={i}{space}{season}'
                    web2 = f'https://www.soccerstats.com/homeaway.asp?league={i}{space}{season}'
                    web3 = f'https://www.soccerstats.com/matchlist.asp?league={i}{space}{season}'
                    overall_list.append(web)
                    home_away_list.append(web2)
                    matches.append(web3)
                if table == 'overall':
                    return overall_list
                if table == "home/away":
                    return home_away_list
                if table == 'matches':
                    return matches
            if date == 'current':
                space = ''
                season = ""
                for i in comp_list:
                    web = f'https://www.soccerstats.com/latest.asp?league={i}{space}{season}'
                    web2 = f'https://www.soccerstats.com/homeaway.asp?league={i}{space}{season}'
                    web3 = f'https://www.soccerstats.com/matchlist.asp?league={i}{space}{season}'
                    overall_list.append(web)
                    home_away_list.append(web2)
                    matches.append(web3)
                if table == 'overall':
                    return overall_list
                if table == "home/away":
                    return home_away_list
                if table == 'matches':
                    return matches

In [51]:
class scraping:
    def ELO_DF(season):
        DF_list = DATA_SOURCE.ELO_source(season)
        full_elo_table = []
        empty_tables = []
        name = 'ELO'
        for i in DF_list:
            web = i
            country = i.partition('=')[2].partition('&')[0]
            driver = webdriver.Chrome('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\Python\\chromedriver.exe')
            driver.get(web)
            html = driver.page_source
            try:
                tables = pd.read_html(html, header=0)
            except ValueError:
                empty_tables.append(i)
                print(i, 'Table not found')

                continue
            driver.close()
            n = 1
            while True:
                try:
                    data = tables[n]
                    l = data['#']
                    break
                except:
                    #print('Try with 2')
                    n = 2
                    try:
                        data = tables[n]
                        l = data['#']
                        break
                    except:
                        #print('Try with 3')
                        n=3
                        try:
                            data = tables[n]
                            l = data['#']
                            break
                        except:
                            #print('Try with 4')
                            n=4
            #print(i, "Scan complete")    
            
            data.rename(columns={
                '#':'Elo_position',"Club":'Team',"Form (last 6)":'F1','Unnamed: 3':'F2','Unnamed: 4':'F3','Unnamed: 5':'F4','Unnamed: 6':'F5','Unnamed: 7':'F6',
                 "Record:Season":"season_record","All time":"record","+/-:1M":"month_diff","Season-to-date +/-:Total":"season_to_date_total"                
                },inplace=True)
            full_elo_table.append(data)
            full_DF = concat(full_elo_table)
        print("ELO scan complete")
        return full_DF 
    
      
    def overall(season, import_csv:bool, date):
            web_list = DATA_SOURCE.soccer_stats_data(season, 'overall', date)
            name1 = 'overall'
            id=1
            empty_tables = []
            key_errors = []
            full_table_list = []
            excel_name1 = f'{season}_{name1}'
            for i in web_list:
                web = i
                name = "overall"
                country1 = i.partition('=')[2].partition('_')[0]
                if '2' in country1: country = country1.partition('2')[0] 
                elif '3' in country1: country = country1.partition('3')[0] 
                elif '4' in country1: country = country1.partition('4')[0] 
                elif '5' in country1: country = country1.partition('5')[0] 
                else: country = country1
                Competition = i.partition('=')[2].partition('_')[0]
                season_id = f'{season}_{Competition}_{country}'
                excel_name = f'{name}_{Competition}_{country}_{season}'
                comp_id = id
                id = id+1
                driver = webdriver.Chrome('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\Python\\chromedriver.exe')
                driver.get(web)
                html = driver.page_source
                try:
                    tables = pd.read_html(html, header=0)
                except ValueError:
                    empty_tables.append(i)
                    print(i, 'Table not found')

                    continue
                driver.close()
                n = functions.loop_tables(tables)
                data0 = tables[n]
                data = tables[n].drop(data0[data0['W'].isnull()].index)
                #print(i, ' Done')
                try:                     
                    data.drop(labels="Form",axis=1, inplace=True,)
                except KeyError:
                    print(i, "Form not found in axis")
                    key_errors.append(i)
                    continue
                try:
                    data.drop(labels="last 8",axis=1, inplace=True,)
                except KeyError:
                    print(i, "last 8 not found")
                    pass
                data.drop(labels="CS",axis=1, inplace=True,)
                data.drop(labels="FTS",axis=1, inplace=True,)
                data.rename(columns={"Unnamed: 0":"Rk","Unnamed: 1":"Team","GP":"MP"
                },inplace=True)
                data.insert(0, "Country", f"{country}", False)
                data.insert(0,"season", f"{season}", False)
                data.insert(0, "Competition",f"{Competition}", False)
                data.insert(0, "Comp_id",f"{comp_id}", False)
                data.insert(0, "season_id",f"{season_id}", False)
                data.insert(16, "xG","", False)
                data.insert(17, "xGA","", False)
                data.insert(18, "xGD","", False)
                data.insert(19, "xGD/90","", False)
                data.insert(20, "Attendance","", False)
                data.insert(21, "Top Team Scorer","", False)
                data.insert(22, "Goalkeeper","", False)
                data.insert(23, "Notes","", False)
                n=n+1
                if import_csv == True:
                    #data.to_csv(os.path.join("C:\\All fotball project\\soccerstats\\overall_2021_2022",f'{excel_name}_data.csv'),index=False)
                    print(i, "export ok")
                if import_csv == False:
                    full_table_list.append(data)
                    #print(i, 'append ok')

                    continue
            if full_table_list == []:
                return 'CSV export succes'
            if full_table_list != []:
                data = pd.concat(full_table_list)
                #data.to_csv(os.path.join("C:\\All fotball project\\Data\\2021-2022\\Overall",f'{excel_name1}_data.csv'),index=False)
                print('Concat OV data success')
                print('CSV full table success')
                print(empty_tables, ':Tables not found')
                return data
    def home_away(season, import_csv:bool, date):
            web_list = DATA_SOURCE.soccer_stats_data(season, 'home/away',date)
            name1 = 'home_away'
            id=1
            empty_tables = []
            full_table_list = []
            excel_name1 = f'{season}_{name1}'
            for i in web_list:
                web = i
                name = 'home_away'
                country1 = i.partition('=')[2].partition('_')[0]
                if '2' in country1: country = country1.partition('2')[0] 
                elif '3' in country1: country = country1.partition('3')[0] 
                elif '4' in country1: country = country1.partition('4')[0] 
                elif '5' in country1: country = country1.partition('5')[0] 
                else: country = country1
                Competition = i.partition('=')[2].partition('_')[0]
                season_id = f'{season}_{Competition}_{country}'
                excel_name = f'{name}_{Competition}_{country}_{season}'
                comp_id = id
                id = id+1
                driver = webdriver.Chrome('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\Python\\chromedriver.exe')
                driver.get(i)
                html = driver.page_source
                try:
                    tables = pd.read_html(html, header = 0)
                except ValueError:
                    empty_tables.append(i)
                    print(i, 'Table not found')
                driver.close()
                n1="home"
                n2="away"
                
                n = functions.loop_tables(tables)
                data1 = tables[n] 
                data1.rename(columns={"Unnamed: 0":"Rk","Unnamed: 1":"Team","GP":f"{n1}_MP",'W':f"{n1}_Win",'D':f"{n1}_Draw",'L':f"{n1}_Losse",'GF':f"{n1}_F",'GA':f"{n1}_A",'GD':f"{n1}_Diff",'Pts':f"{n1}_Pts"
                },inplace=True)
                ppg1 = data1["home_Pts"]/data1['home_MP']
                ppg1_2 = list(np.around(np.array(ppg1),2))
                data1.insert(10, "home_PPG", ppg1_2, False)
                data1.insert(11, "xG", "", False)
                data1.insert(12, "xGA", "", False)
                data1.insert(13, "xGD", "", False)
                data1.insert(14, "xGD/90", "", False)
                nn = functions.loop_tables(tables)
                df2 = tables[nn]
                data2 = pd.DataFrame(df2)
                data2.rename(columns={"Unnamed: 0":"Rk","Unnamed: 1":"Team","GP":f"{n2}_MP",'W':f"{n2}_Win",'D':f"{n2}_Draw",'L':f"{n2}_Losse",'GF':f"{n2}_F",'GA':f"{n2}_A",'GD':f"{n2}_Diff",'Pts':f"{n2}_Pts"
                },inplace=True)
                ppg2 = data2["away_Pts"]/data2['away_MP']
                ppg2_2 = list(np.around(np.array(ppg2),2))
                data2.insert(10, "away_PPG", ppg2_2, False)
                data2.insert(11, "xG", "", False)
                data2.insert(12, "xGA", "", False)
                data2.insert(13, "xGD", "", False)
                data2.insert(14, "xGD/90", "", False)

                data = data1.merge(data2, how="left",left_on='Team',right_on="Team")
                data.drop(labels="Rk_y",axis=1, inplace=True,)
                data.rename(columns={"Rk_x":"Rk"},inplace=True)
                if import_csv == True:
                    #data.to_csv(os.path.join("C:\\All fotball project\\soccerstats\\home_away_2021_2022",f'{excel_name}_data.csv'),index=False)
                    print(i, 'export ok')
                if import_csv == False:
                    full_table_list.append(data)
                    #print(i, 'append ok')
            if full_table_list == []:
                return 'CSV export succes'
            if full_table_list != []:
                data = pd.concat(full_table_list)
                #data.to_csv(os.path.join("C:\\All fotball project\Data\\2021-2022\\home_away",f'{excel_name1}_data.csv'),index=False)
                print('Concat home/away data success')
                print('CSV full table success')
                print(empty_tables, ':Tables not found')
                return data
    def matches(season, import_csv:bool, date):
            web_list = DATA_SOURCE.soccer_stats_data(season, 'matches',date)
            name1 = 'matches'
            id=1
            empty_tables = []
            full_table_list = []
            excel_name1 = f'{season}_{name1}'
            for i in web_list:
                web = i
                name = 'matches'
                country1 = i.partition('=')[2].partition('_')[0]
                if '2' in country1: country = country1.partition('2')[0] 
                elif '3' in country1: country = country1.partition('3')[0] 
                elif '4' in country1: country = country1.partition('4')[0] 
                elif '5' in country1: country = country1.partition('5')[0] 
                else: country = country1
                Competition = i.partition('=')[2].partition('_')[0]
                season_id = f'{season}_{Competition}_{country}'
                excel_name = f'{name}_{Competition}_{country}_{season}'
                comp_id = id
                id = id+1
                driver = webdriver.Chrome('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\Python\\chromedriver.exe')
                driver.get(web)
                html = driver.page_source
                try:
                    tables = pd.read_html(html, header=0)
                except ValueError:
                    empty_tables.append(i)
                    print(i, 'Table not found')
                    continue
                driver.close()
                n = functions.loop_matches(tables)
                data = tables[n] 
                data.rename(columns={"Local":"home_team","Visitor":'away_team',"Outcome":"result"
                                     },inplace=True)
                data = data[data['result'].isin(['Draw', 'Home win', 'Away win'])]
                if import_csv == True:
                    #data.to_csv(os.path.join("C:\\All fotball project\\soccerstats\\home_away_2021_2022",f'{excel_name}_data.csv'),index=False)
                    print(i, 'export ok')
                if import_csv == False:
                    full_table_list.append(data)
                    #print(i, 'append ok')
            if full_table_list == []:
                return 'CSV export succes'
            if full_table_list != []:
                data = pd.concat(full_table_list)
                #data.to_csv(os.path.join("C:\\All fotball project\Data\\2021-2022\\home_away",f'{excel_name1}_data.csv'),index=False)
                print('Concat matches data success')
                print('CSV full table success')
                print(empty_tables, ':Tables not found')
                return data    

In [35]:
class replaces:
    def short_names(data, step):
        if step == 1:
            short = lists.clubs_3()
            new_team_name = []
            for i in data:
                # Store name, for if no match found
                i = str(i)
                name = str(i)
                for j in short:
                    if j in i:
                        # Reformat name if match
                        name = i.replace(f' {j}', "").replace(f'{j} ', "")
                new_team_name.append(name)
            return new_team_name
        if step == 2:
            short = lists.nums()
            new_team_name = []
            for i in data:
                # Store name, for if no match found
                i = str(i)
                name = str(i)
                for j in short:
                    if j in i:
                        # Reformat name if match
                        name = i.replace(f' {j}', "").replace(f'{j} ', "").replace(f'{j}',"")
                new_team_name.append(name)
            return new_team_name
        if step == 3:
            short = lists.clubs_2()
            new_team_name = []
            for i in data:
                # Store name, for if no match found
                i = str(i)
                name = str(i)
                for j in short:
                    if j in i:
                        # Reformat name if match
                        name = i.replace(f' {j}', "").replace(f'{j} ', "")
                new_team_name.append(name)
            return new_team_name
        if step == 4:
            short = lists.names()
            new_team_name = []
            for i in data:
                # Store name, for if no match found
                i = str(i)
                name = str(i)
                for j in short:
                    if "LASK" in i:
                        continue
                    if j in i:
                        # Reformat name if match
                        name = i.replace(f'{j}', "")
                new_team_name.append(name)
            return new_team_name

In [44]:
class procedures:
    def rename_teams(data, ha):
        if ha == 0:
            new_short_name = []
            names = data['short_name']
            for i in names:
                i = str(i)
                name = lists.teams(i, 1)
                name1 = lists.teams(name, 2)
                name2 = lists.teams(name1, 3)
                new_short_name.append(name2)

            if 'short_name' in data.columns:
                del data['short_name']
            data.insert(0,"short_name",new_short_name,True)
            return data
        if ha == 1:
            new_short_home_name = []
            new_short_away_name = []
            names1 = data['short_name_home']
            names2 = data['short_name_away']
            for i, j in zip(names1, names2):
                i = str(i)
                j = str(j)
                name = lists.teams(i, 1)
                name1 = lists.teams(name, 2)
                name2 = lists.teams(name1, 3)
                new_short_home_name.append(name2)
                name_a = lists.teams(j, 1)
                name1_a = lists.teams(name_a, 2)
                name2_a = lists.teams(name1_a, 3)
                new_short_away_name.append(name2_a)
            if 'short_name_home' in data.columns:
                del data['short_name_home']
            if 'short_name_away' in data.columns:
                del data['short_name_away']
            data.insert(0,"short_name_home",new_short_home_name,True)
            data.insert(0,"short_name_away",new_short_away_name,True)
            return data
    
    
    def repl(data):
        data1 = replaces.short_names(data, 1)
        data2 = replaces.short_names(data1,2)
        data3 = replaces.short_names(data2,3)
        data4 = replaces.short_names(data3,4)
        return data4
            
    def elo_team_update(data):
        if 'short_name' in data.columns:
            del data['short_name']
        if "team_id" in data.columns:
            del data['team_id']
        values = data['Team']
        team_name = []
        new_team_name = procedures.repl(values)

        data.insert(0,"short_name",new_team_name,True)
        return data
    def teams(data):
        if 'short_name' in data.columns:
            del data['short_name']
        if "team_id" in data.columns:
            del data['team_id']
        #values = data['common_name']
        values = data['Team']
        team_name = []
        new_team_name = procedures.repl(values)
        
    
        data.insert(0,"short_name",new_team_name,True)
 
        return data
    def players(data):
        if 'short_name' in data.columns:
            del data['short_name']
        if "team_id" in data.columns:
            del data['team_id']
        values = data['Current Club']
        team_name = []
        new_team_name = procedures.repl(values)
        
    
        data.insert(0,"short_name",new_team_name,True)

        return data
        
        
    def matches(data):
        if 'short_name_home' in data.columns:
            del data['short_name_home']
        if 'short_name_away' in data.columns:
            del data['short_name_away']
        if "team_id" in data.columns:
            del data['team_id']
        tm = []
        tm2 = []
        
        values1 = data['home_team']
        values2 = data['away_team']
        home_team_name = procedures.repl(values1)
        away_team_name = procedures.repl(values2)
            
        
        data.insert(0,"short_name_home",home_team_name,True)
        data.insert(0,"short_name_away",away_team_name,True)

        return data
    
    def create_id(data):
        id = []
        n=1
        for i in data['short_name']:
            id.append(n)
            n=n+1
        data.insert(0,'team_id',id,True)
        return data

In [8]:
class DFOP:
    def Elo_DF():
        ELO = scraping.ELO_DF('2022-2023')
        return ELO
    def data_from_directory(csv:int):
            data = pd.read_csv(f'{csv}.csv', encoding = 'utf-8', sep = ',')
            return data
    def export_csv(loc:str, data, name):
        location = loc
        data.to_csv(os.path.join(location,f'{name}_data.csv'),index=False)
        
        
    def get_csv_from_folder(path):
        

        # read all the files with extension .csv
        filenames = glob.glob(path + "\*.csv")
        #print('File names:', filenames)
        li = []
        # for loop to iterate all csv files
        for file in filenames:
           # reading csv files
           #print("\nReading file = ",file)
           #print(pd.read_csv(file))
           li.append(file)
        return li
    def concat_DF(path):
        li = DFOP.get_csv_from_folder(path)
        DF_list = []

        for j in li:
            tb = pd.read_csv(j, encoding = 'utf-8', sep = ',')
            df = pd.DataFrame(tb)
            file_name = j.partition("-teams")[0].partition("Teams")[2]
            file_name = j.partition("-matches")[0].partition("Matches")[2]
            file_name = j.partition("-matches")[0].partition("Players")[2]
            file_name = j.partition("-matches")[0].partition("Season")[2]
            df.insert(0,"file_name",file_name,False)
            DF_list.append(df)
            
        DF = concat(DF_list)
        return DF

In [21]:
class SQL:
    def create_DB(DB_name):
        try:
            conn = msql.connect(host='localhost', user='polonez',  
                        password='lacosanostra2')#give ur username, password
            if conn.is_connected():
                cursor = conn.cursor()
                cursor.execute(f"CREATE DATABASE {DB_name}")
            print("Database is created")
        except Error as e:
            print("Error while connecting to MySQL", e)
    
    def select_DB(DB, query):
        try:
            conn = msql.connect(host='localhost', database=DB, user='polonez', password='lacosanostra2')#give ur username, password
            if conn.is_connected():
                cursor = conn.cursor()
                cursor.execute("select database();")
                record = cursor.fetchone()
                print("You're connected to database: ", record)
                cursor.execute(query)
                data_list = cursor.fetchall()
                column_names = [i[0] for i in cursor.description]
                data = pd.DataFrame(data_list, columns = column_names)
        except Error as e:
                    print("Error while connecting to MySQL", e)
            
        return data 
    
    def load_data(DB, table, DF, truncate):
        try:
            conn = msql.connect(host='localhost', database=DB, user='polonez', password='lacosanostra2')#give ur username, password
            if conn.is_connected():
                cursor = conn.cursor()
                cursor.execute("select database();")
                record = cursor.fetchone()
                table_name = table
                print("You're connected to database: ", record)
                if truncate == True:
                    cursor.execute(f'TRUNCATE TABLE {table};')
                    print('Truncate')
                else:
                    pass
                v = []
                for j in DF.keys():
                        v.append(j)            
                li = []
                for i in range(len(v)):
                    li.append('%s')
                #print("Table is created....")
                for i,row in DF.iterrows():
                    print("Data loading........")
                    sql = f"INSERT INTO {DB}.{table} VALUES {tuple(li)}".replace("'","")
                    cursor.execute(sql, tuple(row))
                    print("Record inserted")
                    conn.commit()
                cursor.execute(f"UPDATE {table_name} SET short_name = trim(short_name) ")
        except Error as e:
                    print("Error while connecting to MySQL", e)
                    
                                        
    def select_id():
        try:
            conn = msql.connect(host='localhost', database="values", user='root', password='lacosanostrasql')
            if conn.is_connected():
                cursor = conn.cursor()
                cursor.execute("select database();")
                record = cursor.fetchone()
                print("You're connected to database: ", record)
                cursor.execute('SELECT * FROM values.team_id;')
                data_list = cursor.fetchall()
                column_names = [i[0] for i in cursor.description]
                DF = pd.DataFrame(data_list, columns = column_names)
        except Error as e:
                    print("Error while connecting to MySQL", e)
        return DF

In [22]:
SQL.select_id()

You're connected to database:  ('values',)


,team_id,short_name
0,1,Liverpool
1,2,Manchester City
2,3,Chelsea
3,4,Arsenal
4,5,Tottenham Hotspur
...,...,...
684,972,Yeni Malatyaspor
685,974,Altinordu
686,976,Tuzlaspor
687,978,Adanaspor


In [32]:
class current_data:
    def ELO():
        ELO_Data = scraping.ELO_DF('2022-2023')
        ELO_update = procedures.elo_team_update(ELO_Data)
        ELO = procedures.rename_teams(ELO_update, 0)
        id = SQL.select_id()
        DF = pd.merge(ELO, id, how = 'inner', left_on="short_name", right_on="short_name")
        first_column = DF.pop('team_id')
        DF.insert(0, 'team_id', first_column)
        DF1 = DF.astype(object).where(pd.notnull(DF), None)
        ln = len(DF['short_name'])
        print("Done: ", ln," records inserted")
        return DF1
    def current_stat(df):
        if df == 'OV':
            OV_data = scraping.overall('2023', False, 'current')
            OV_update = procedures.teams(OV_data)
            OV = procedures.rename_teams(OV_update, 0)
            id = SQL.select_id()
            DF = pd.merge(OV, id, how = 'inner', left_on="short_name", right_on="short_name")
            DF1 = DF.astype(object).where(pd.notnull(DF), None)
            ln = len(DF['short_name'])
            print("Done: ", ln," records inserted")
            return DF1
        if df == 'HA':
            home_away_data = scraping.home_away('2023', False, 'current')
            home_away_update = procedures.teams(home_away_data)
            HA = procedures.rename_teams(home_away_update, 0)
            id = SQL.select_id()
            DF = pd.merge(HA, id, how = 'inner', left_on="short_name", right_on="short_name")
            DF1 = DF.astype(object).where(pd.notnull(DF), None)
            ln = len(DF['short_name'])
            print("Done: ", ln," records inserted")
            return DF1
        if df == 'MT':
            matches = scraping.matches('2023', False, 'current')
            home_away_update = procedures.matches(matches)
            HA = procedures.rename_teams(home_away_update, 1)
            id = SQL.select_id()
            DF_home = pd.merge(HA, id, how = 'inner', left_on="short_name_home", right_on="short_name")
            DF = pd.merge(DF_home, id, how = 'inner', left_on="short_name_away", right_on="short_name")
            DF1 = DF.astype(object).where(pd.notnull(DF), None)
            ln = len(DF['short_name_home'])
            print("Done: ", ln," records inserted")
            return DF1
        
    def DF_current():
        start_time = time.time()
        ELO = current_data.ELO()
        OV = current_data.current_stat('OV')
        HA = current_data.current_stat('HA')
        MT = current_data.current_stat('MT')
        print('Scrap Data Complete')
        print("--- %s seconds ---" % (time.time() - start_time))
        return ELO, OV, HA, MT

In [60]:
class load_data:
    def load_to_SQL():
        start_time = time.time()

        ELO = current_data.ELO()
        DFOP.export_csv('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\DATA',ELO,"ELO")
        print('CSV export complete')

        OV = current_data.current_stat('OV')
        DFOP.export_csv('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\DATA',OV,"overall")
        print('CSV export complete')

        HA = current_data.current_stat('HA')
        DFOP.export_csv('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\DATA',HA,"home_away")
        print('CSV export complete')

        MT = current_data.current_stat('MT')
        DFOP.export_csv('C:\\Users\\darju\\OneDrive\\Stalinis kompiuteris\\Project\\DATA',MT,"matches")
        print('CSV export complete')#Get DATA


        #load to SQL
        SQL.load_data('elo_data','elo_data',ELO,True) 
        SQL.load_data('current_data','overall_data',ELO,True)
        SQL.load_data('current_data','home_away_data',HA,True)
        SQL.load_data('current_data','matches_data',MT,True)
        
        print('Data load complete')
        print("--- %s seconds ---" % (time.time() - start_time))